In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# E-commerce Analytics - Notebook de Exemplo\n",
    "\n",
    "Este notebook demonstra como usar o sistema de análise de e-commerce diretamente do Jupyter.\n",
    "\n",
    "**Autor:** Vitor Oliveira  \n",
    "**Data:** 2025-05-26"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Configuração do Ambiente\n",
    "\n",
    "Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Adicionar o diretório raiz ao path para importar os módulos\n",
    "import sys\n",
    "import os\n",
    "sys.path.append(os.path.abspath('..'))\n",
    "\n",
    "# Importar as bibliotecas necessárias\n",
    "import polars as pl\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import plotly.express as px\n",
    "from datetime import datetime\n",
    "\n",
    "# Importar os módulos do projeto\n",
    "from src.controllers.analise_controller import AnaliseController\n",
    "from src.models.obter_dados_ecommerce import ObterDadosEcommerce\n",
    "from src.models.ecommerce_model import EcommerceModel\n",
    "from src.views.powerbi_exporter import PowerBIExporter\n",
    "\n",
    "# Configurar visualização\n",
    "%matplotlib inline\n",
    "plt.style.use('ggplot')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "print(\"Ambiente configurado com sucesso!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Geração de Dados Sintéticos\n",
    "\n",
    "Vamos gerar alguns dados sintéticos para análise:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Inicializar o gerador de dados\n",
    "gerador = ObterDadosEcommerce()\n",
    "\n",
    "# Gerar dados sintéticos (2000 registros)\n",
    "arquivo_dados = gerador.gerar_dados_sinteticos(2000)\n",
    "\n",
    "# Carregar os dados gerados\n",
    "dados_brutos = pl.read_csv(arquivo_dados)\n",
    "\n",
    "# Exibir as primeiras linhas\n",
    "dados_brutos.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Processamento de Dados\n",
    "\n",
    "Agora vamos processar esses dados:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Inicializar o modelo\n",
    "modelo = EcommerceModel()\n",
    "\n",
    "# Processar os dados\n",
    "dados_processados = modelo.process_sales_data(dados_brutos)\n",
    "\n",
    "# Verificar o resultado\n",
    "print(f\"Formato dos dados processados: {dados_processados.shape}\")\n",
    "dados_processados.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Análise de Vendas por Período\n",
    "\n",
    "Vamos analisar as vendas por período usando o controlador:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Inicializar o controlador\n",
    "controller = AnaliseController()\n",
    "\n",
    "# Analisar vendas por período\n",
    "analises = controller.analisar_vendas_por_periodo(\n",
    "    dados_processados,\n",
    "    coluna_data='date',\n",
    "    coluna_valor='total_value'\n",
    ")\n",
    "\n",
    "# Exibir resultados\n",
    "for nome, df in analises.items():\n",
    "    print(f\"\\n{nome.upper()}:\")\n",
    "    display(df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Visualização de Dados\n",
    "\n",
    "Vamos criar algumas visualizações para os dados processados:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Converter para pandas para facilitar a visualização\n",
    "df_pandas = dados_processados.to_pandas()\n",
    "\n",
    "# 1. Vendas por categoria\n",
    "vendas_por_categoria = df_pandas.groupby('product_category')['total_value'].sum().sort_values(ascending=False)\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "vendas_por_categoria.plot(kind='bar')\n",
    "plt.title('Total de Vendas por Categoria', fontsize=16)\n",
    "plt.xlabel('Categoria')\n",
    "plt.ylabel('Valor Total de Vendas (R$)')\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 2. Vendas por método de pagamento\n",
    "vendas_por_pagamento = df_pandas.groupby('payment_method')['total_value'].sum().sort_values(ascending=False)\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "vendas_por_pagamento.plot(kind='pie', autopct='%1.1f%%')\n",
    "plt.title('Distribuição de Vendas por Método de Pagamento', fontsize=16)\n",
    "plt.ylabel('')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 3. Vendas por estado (mapa de calor com Plotly)\n",
    "vendas_por_estado = df_pandas.groupby('state')['total_value'].sum().reset_index()\n",
    "\n",
    "fig = px.choropleth_mapbox(\n",
    "    vendas_por_estado, \n",
    "    locations='state', \n",
    "    color='total_value',\n",
    "    color_continuous_scale=\"Viridis\",\n",
    "    range_color=(0, vendas_por_estado['total_value'].max()),\n",
    "    mapbox_style=\"carto-positron\",\n",
    "    zoom=3, \n",
    "    center={\"lat\": -15.8, \"lon\": -47.9},\n",
    "    opacity=0.5,\n",
    "    labels={'total_value': 'Valor Total (R$)'}\n",
    ")\n",
    "\n",
    "fig.update_layout(\n",
    "    title_text='Vendas por Estado', \n",
    "    height=600\n",
    ")\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 4. Tendência de vendas ao longo do tempo\n",
    "df_pandas['date'] = pd.to_datetime(df_pandas['date'])\n",
    "vendas_por_dia = df_pandas.groupby(df_pandas['date'].dt.date)['total_value'].sum().reset_index()\n",
    "\n",
    "fig = px.line(\n",
    "    vendas_por_dia, \n",
    "    x='date', \n",
    "    y='total_value',\n",
    "    labels={'date': 'Data', 'total_value': 'Valor Total (R$)'},\n",
    "    title='Tendência de Vendas ao Longo do Tempo'\n",
    ")\n",
    "\n",
    "fig.update_layout(\n",
    "    xaxis_title='Data',\n",
    "    yaxis_title='Valor Total de Vendas (R$)',\n",
    "    height=500\n",
    ")\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Exportação para Power BI\n",
    "\n",
    "Por fim, vamos exportar os dados para uso no Power BI:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Inicializar o exportador\n",
    "exportador = PowerBIExporter()\n",
    "\n",
    "# Exportar dados processados\n",
    "caminho_exportado = exportador.export_to_csv(dados_processados, 'dados_ecommerce_notebook')\n",
    "print(f\"Dados exportados para: {caminho_exportado}\")\n",
    "\n",
    "# Exportar análises agregadas\n",
    "for nome, df in analises.items():\n",
    "    caminho = exportador.export_to_csv(df, f'analise_{nome}_notebook')\n",
    "    print(f\"Análise '{nome}' exportada para: {caminho}\")\n",
    "\n",
    "print(\"\\nTodos os dados foram exportados com sucesso e estão prontos para importação no Power BI!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Conclusão\n",
    "\n",
    "Neste notebook, demonstramos o fluxo completo de análise de dados de e-commerce:\n",
    "\n",
    "1. Geração de dados sintéticos\n",
    "2. Processamento dos dados brutos\n",
    "3. Análise de vendas por período\n",
    "4. Criação de visualizações\n",
    "5. Exportação para Power BI\n",
    "\n",
    "Este fluxo pode ser adaptado para dados reais de vendas, permitindo análises mais complexas e insights valiosos para o negócio."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.13"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}